# Informations sur la base de données.

#### Buts de l'analyse

#### Étapes de traitement des données

#### Importation des librairies utilisées.

In [1]:
import os

import sklearn
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

import scipy.stats as stats

%matplotlib inline
os.environ['KMP_DUPLICATE_LIB_OK']='True'
# Pour la reproductibilité des résultats
np.random.seed(43)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore', '.*do not.*', )
warnings.warn('Do not show this message')

# some nice colors from http://colorbrewer2.org/
COLOR1 = '#7fc97f'
COLOR2 = '#beaed4'
COLOR3 = '#fdc086'
COLOR4 = '#ffff99'
COLOR5 = '#386cb0'

sns.set(rc={'figure.figsize':(9, 4)})
sns.set_theme(context="notebook", style="white", palette="Set2")

## Recommandations 

### Analyse des forme


* **Ligne et colonne: (6498063, 7)**

niveau de detail, une ligne par :
                  Géographie, 
                  Classification nationale des professions(CNP4),
                  Sexe,
                  Durée de l'emploi,	
                  Période de référence, mensuelle

* **Type de variables: 10 variables de type numérique et 4 variables qualitative.**

    * Géographie : Canada, Province ou territoire
    * Ratio du salaire hebdomadaire moyen entre les femmes et les hommes	
    * Ratio du salaire hebdomadaire médian entre les femmes et les hommes
    * Ratio du salaire horaire moyen entre les femmes et les hommes	
    * Ratio du salaire horaire médian entre les femmes et les hommes	
    * Salaire hebdomadaire moyen	
    * Salaire hebdomadaire médian	
    * Salaire horaire moyen	
    * Salaire horaire médian	
    * Total des employés, tous les salaires
    * Classification nationale des professions(CNP4)
    * Sexe
    * Groupe d’âge
    * Période de référence : annuelle

La repartition des individus est egale pour toutes les modalites des variables Sexe, CNP, Groupe d'age

* **Type des valeurs manquantes**  

    * 25% des valeurs a 0 pour les variables 
        * Salaire hebdomadaire moyen	
        * Salaire hebdomadaire médian	
        * Salaire horaire moyen	
        * Salaire horaire médian
    
Il y a 48277(25%) des enregsitrements dont la valeur du salaire hebdomadaire moyen est egal a 0.

Repartition: 

CNP: 
* 2049 (1.37%), Superviseurs/superviseures et métiers techniques dans les ressources naturelles, l'agriculture et la production connexe, 
* 2106 (1.25%), Personnel des services de protection public de première ligne
* 2049	(1.22%) Cadres supérieurs/cadres supérieures [00]		

Regions:

* 4.60% (7743) dans Île-du-Prince-Édouard et Terre-Neuve-et-Labrador	
* 4.42% (5986)	Nouveau-Brunswick et Nouvelle-Écosse
* 1.56% (2628) Québec	   
    
* **Valeurs extremes **

On va supposer, pour l'exemple, que la base de données contient 0,5% de valeurs aberrantes de toutes sortes. Ce n'est pas exagéré. Cela représente $0,005*168300 \approx 841$ valeurs.

On utilise la méthode de la [forêt d'isolation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html) pour détecter et éliminer un maximum de 0,5% des valeurs possiblement aberrantes.  
    
### Analyse de structure

* Calculer difference de Salaire horaire moyen entre  Salaire horaire moyen (homme)-  Salaire horaire moyen(femme)
* Conserver seulement les variables :  
    * Géographie : Canada, Province ou territoire,
    * Salaire horaire moyen	
    * Classification nationale des professions(CNP4)
    * Sexe
    * Groupe d’âge
    * Période de référence : annuelle


### Nettoyage des donnees: Recommandations


* Filtrer la modalite "Les deux sexes" de la variable **Sexe**
* Filtrer la modalite "15 ans et plus" de la variable **Groupe d'âge**
* Filtrer la modalite "'Canada'" de la variable **Region**


# Partie I: Prétraitement des données.

## Lecture des bases de données en format CSV.

Les données sont écrites dans un DataFrame de la librairie Pandas.

In [2]:
def load_db():
    duree_emploi_CNP=pd.read_csv("duree_emploi_CNP",sep=",")
    return duree_emploi_CNP#,postes_vacants_CNP,caractéristiques_population_active,heures_habituellement_travaille,heures_effectivement_travaille,personnes_avec_plus_un_emploi,raison_quitter_emploi,absence_travail_temps_plein

In [3]:
duree_emploi_CNP=load_db()

## Explorer les données

In [4]:
duree_emploi_CNP.head()

,Année/Mois,Régions,CNP,Sexe,Durée de l'emploi,Mois,Personnes
0,1987-01,Alberta,"Affaires, finance et administration [1]",Femmes,1 à 3 mois,NaN,14.1
1,1987-01,Alberta,"Affaires, finance et administration [1]",Femmes,121 à 240 mois,NaN,19.7
2,1987-01,Alberta,"Affaires, finance et administration [1]",Femmes,13 à 60 mois,NaN,53.4
3,1987-01,Alberta,"Affaires, finance et administration [1]",Femmes,241 mois ou plus,NaN,7.1
4,1987-01,Alberta,"Affaires, finance et administration [1]",Femmes,4 à 6 mois,NaN,12.4


In [5]:
duree_emploi_CNP.tail()

,Année/Mois,Régions,CNP,Sexe,Durée de l'emploi,Mois,Personnes
6498058,2022-09,Île-du-Prince-Édouard,Vente et services [6],Les deux sexes,4 à 6 mois,NaN,2.9
6498059,2022-09,Île-du-Prince-Édouard,Vente et services [6],Les deux sexes,61 à 120 mois,NaN,1.8
6498060,2022-09,Île-du-Prince-Édouard,Vente et services [6],Les deux sexes,7 à 12 mois,NaN,1.6
6498061,2022-09,Île-du-Prince-Édouard,Vente et services [6],Les deux sexes,Durée moyenne,83.8,NaN
6498062,2022-09,Île-du-Prince-Édouard,Vente et services [6],Les deux sexes,"Emploi total, tous les mois",NaN,16.5


### Dimensions du dataset

In [6]:
print("Les dimensions du jeux des donnees: ", duree_emploi_CNP.shape)
print() 

Les dimensions du jeux des donnees:  (6498063, 7)



In [8]:
# Size of our data set
## le nombre de lignes multiplié par le nombre de colonnes si DataFrame.
print("Nombre d’éléments:", duree_emploi_CNP.size)
print()

Nombre d’éléments: 45486441



Le nombre d'individus est égale à 6498063 et le nombre de descripteurs est égale à 7

### Type des donnees

In [9]:
duree_emploi_CNP.dtypes

Année/Mois            object
Régions               object
CNP                   object
Sexe                  object
Durée de l'emploi     object
Mois                 float64
Personnes            float64
dtype: object

In [10]:
duree_emploi_CNP.dtypes.value_counts()

object     5
float64    2
dtype: int64

Le jeu des données a 2 variables de type numérique et 5 variables qualitative.

In [11]:
duree_emploi_CNP.describe()

,Mois,Personnes
count,722007.000000,5.776056e+06
mean,95.811712,2.788090e+01
std,42.514423,2.198379e+02
min,0.000000,0.000000e+00
25%,70.100000,0.000000e+00
50%,95.500000,2.000000e+00
75%,119.500000,1.020000e+01
max,474.000000,1.996600e+04


In [13]:
duree_emploi_CNP.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6498063 entries, 0 to 6498062
Data columns (total 7 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Année/Mois         object 
 1   Régions            object 
 2   CNP                object 
 3   Sexe               object 
 4   Durée de l'emploi  object 
 5   Mois               float64
 6   Personnes          float64
dtypes: float64(2), object(5)
memory usage: 347.0+ MB


In [15]:
duree_emploi_CNP.describe(include=[object])

,Année/Mois,Régions,CNP,Sexe,Durée de l'emploi
count,6498063,6498063,6498063,6498063,6498063
unique,429,11,51,3,9
top,1987-01,Alberta,"Affaires, finance et administration [1]",Femmes,1 à 3 mois
freq,15147,590733,127413,2166021,722007


* La variable Regions contient 11 modalites, la plus frequente est Alberta
* La variable CNP contient 51 modalites, la plus frequente est Affaires, finance et administration [1]
* La variable Sexe contienttient 3 modaliteses, la plus frequente est Femme
* La variable Durée de l'emploi contient 9 modalites, la plus frequente est 1 a 3 mois

In [23]:
duree_emploi_CNP["Durée de l'emploi"].unique()

array(['1 à 3 mois', '121 à 240 mois', '13 à 60 mois', '241 mois ou plus',
       '4 à 6 mois', '61 à 120 mois', '7 à 12 mois', 'Durée moyenne',
       'Emploi total, tous les mois'], dtype=object)

In [24]:
duree_emploi_CNP["Sexe"].unique()

array(['Femmes', 'Hommes', 'Les deux sexes'], dtype=object)

### Affichage du nombre de valeurs manquantes Nans dans le DataFrame.

In [17]:
duree_emploi_CNP.isna().sum()

Année/Mois                 0
Régions                    0
CNP                        0
Sexe                       0
Durée de l'emploi          0
Mois                 5776056
Personnes             722007
dtype: int64

In [20]:
duree_emploi_CNP[duree_emploi_CNP.Mois.notnull()]

,Année/Mois,Régions,CNP,Sexe,Durée de l'emploi,Mois,Personnes
7,1987-01,Alberta,"Affaires, finance et administration [1]",Femmes,Durée moyenne,66.4,NaN
16,1987-01,Alberta,"Affaires, finance et administration [1]",Hommes,Durée moyenne,88.7,NaN
25,1987-01,Alberta,"Affaires, finance et administration [1]",Les deux sexes,Durée moyenne,72.3,NaN
34,1987-01,Alberta,"Arts, culture, sports et loisirs [5]",Femmes,Durée moyenne,42.6,NaN
43,1987-01,Alberta,"Arts, culture, sports et loisirs [5]",Hommes,Durée moyenne,86.8,NaN
...,...,...,...,...,...,...,...
6498025,2022-09,Île-du-Prince-Édouard,Superviseurs/superviseures et métiers techniqu...,Hommes,Durée moyenne,170.1,NaN
6498034,2022-09,Île-du-Prince-Édouard,Superviseurs/superviseures et métiers techniqu...,Les deux sexes,Durée moyenne,182.0,NaN
6498043,2022-09,Île-du-Prince-Édouard,Vente et services [6],Femmes,Durée moyenne,77.6,NaN
6498052,2022-09,Île-du-Prince-Édouard,Vente et services [6],Hommes,Durée moyenne,92.7,NaN


In [21]:
duree_emploi_CNP[duree_emploi_CNP.Mois.notnull()]["Durée de l'emploi"].value_counts()

Durée moyenne    722007
Name: Durée de l'emploi, dtype: int64

In [25]:
duree_emploi_CNP[duree_emploi_CNP.Mois.isnull()]["Durée de l'emploi"].value_counts()

1 à 3 mois                     722007
121 à 240 mois                 722007
13 à 60 mois                   722007
241 mois ou plus               722007
4 à 6 mois                     722007
61 à 120 mois                  722007
7 à 12 mois                    722007
Emploi total, tous les mois    722007
Name: Durée de l'emploi, dtype: int64